In [ ]:
!pip install transformers

In [1]:
print(2)

2


In [2]:
%%time
import os
import deepspeed
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import argparse
model_name="/home/ec2-user/SageMaker/GPTJ/amazon-sagemaker-gptj/finetune/"

#local_rank = int(os.getenv('LOCAL_RANK', '7'))
#world_size = int(os.getenv('WORLD_SIZE', '8'))


model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0,torch_dtype=torch.float16)

generator.model = deepspeed.init_inference(
                                        generator.model,
                                        tensor_parallel={"tp_size": 1},
                                        dtype=torch.half,
                                        replace_method='auto',
                                        max_tokens=2048,
                                        replace_with_kernel_inject=True
                                          
                                          )
torch.cuda.synchronize()

[2023-02-10 19:33:53,790] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed info: version=0.8.0, git-hash=unknown, git-branch=unknown
[2023-02-10 19:33:53,792] [INFO] [logging.py:68:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
Using /home/ec2-user/.cache/torch_extensions/py39_cu116 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ec2-user/.cache/torch_extensions/py39_cu116/transformer_inference/build.ninja...
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module transformer_inference...
Time to load transformer_inference op: 0.48761963844299316 seconds
[2023-02-10 19:33:54,999] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 4096, 'intermediate_size': 16384, 'heads': 16, 'num_hidden_lay

In [ ]:
print(f"model is loaded on device {generator.model.module.device}")

In [4]:
%%time
input_text = "love: "
string = generator(input_text, do_sample=True, max_length=1000,min_length=300, top_k=50, top_p=0.95, temperature=0.9)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


CPU times: user 14.9 s, sys: 0 ns, total: 14.9 s
Wall time: 14.9 s


In [5]:
string

[{'generated_text': 'love:  We must learn to put aside our pride of personality and recognize our common ground as human beings. Our greatest need is the love that will transform us into our common ground, and then that love will bring us together as a people. Only then will we be able to live together in peace. We must stop the constant war. We must stop the endless bickering. We must stop the constant striving for the top. We must stop the constant blaming. And we must finally accept one another for our differences and love each other. That is the hope of the future and the promise of our eternal welfare. That is the hope of life for all of us. That is our hope. That is our promise. That is the hope of humanity. That is the promise of life. That is the hope of the future. That is the promise of life. That is the hope of the future. That is the promise of life. Let us keep these promises. Let us be worthy of the hopes of the past. Let us be worthy of the hopes of the future. Let us wo

In [ ]:
import os
import deepspeed
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import argparse
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('-m',"--model_name", type=str, default='EleutherAI/gpt-j-6B')
    parser.add_argument('--local_rank', type=int, default=-1,
                    help='local rank passed from distributed launcher')
    parser = deepspeed.add_config_arguments(parser)
    args = parser.parse_args()
    model_name = args.model_name

    local_rank = int(os.getenv('LOCAL_RANK', '0'))
    world_size = int(os.getenv('WORLD_SIZE', '1'))

    model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=local_rank,torch_dtype=torch.float16)

    generator.model = deepspeed.init_inference(generator.model,
                                            mp_size=world_size,
                                            dtype=torch.half,
                                            replace_method='auto',
                                            max_tokens=2048,
                        replace_with_kernel_inject=True)
    torch.cuda.synchronize()

    input_text = "DeepSpeed is"
    string = generator(input_text, do_sample=True, max_length=2047,min_length=2047, top_k=50, top_p=0.95, temperature=0.9)

    if not torch.distributed.is_initialized() or torch.distributed.get_rank() == 0:
        print(string)

In [ ]:
from transformers import GPTJForCausalLM, AutoTokenizer
import torch
model_name="/home/ec2-user/SageMaker/GPTJ/amazon-sagemaker-gptj/finetune/"

model = GPTJForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)



In [ ]:
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)
gen_text = tokenizer.batch_decode(gen_tokens)[0]